# Вариант "легкого" решения для предсказания необходимого количества товара в магазинах

Загружаем библиотеки

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
from datetime import timedelta
import numpy as np


Выгружаем данные

Нас интересуют только данные за стабильный период цен на нефть на конце наблюдений. Данные преобразования происходят после долгого анализа исходных данных и применения наработок старших коллег.

In [2]:


dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}

train = pd.read_csv('data/train.csv', usecols=[1,2,3,4], dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 86672217) #Skip dates before 2016-08-01
                    )

In [3]:
train.date.min()

Timestamp('2016-08-01 00:00:00')

In [4]:
train.isnull().sum()

date          0
store_nbr     0
item_nbr      0
unit_sales    0
dtype: int64

In [5]:
train.shape

(38824824, 4)

получаем данные по среднему количеству товара на каждый день недели(ma_dw) и за неделю(ma_wk)

In [6]:
train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek

#Days of Week Means
#By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(
        ['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw').reset_index()
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(
        ['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk').reset_index()

train.drop('dow',axis=1,inplace=True)

для корректной работы в дальнейшем поработаем с индексами

In [7]:
# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
).reset_index()



In [8]:
train.isnull().sum()

date                 0
store_nbr            0
item_nbr             0
unit_sales    43775952
dtype: int64

данные расширились и появились пустые значения

In [9]:
train.shape

(82600776, 4)

In [10]:
del u_dates, u_stores, u_items

подчищаем данные

Рассмотрим средние значения целевой переменной на различных недельных периодах,те получим общее поведение товара за периоды времени.(Почему? Вспоминаем анализ данных, была такая зависимость)

In [11]:
train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
lastdate = train.iloc[train.shape[0]-1].date

#Moving Averages
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(
        ['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais')

for i in [112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')



/tmp/ipykernel_6531/4233960327.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs


In [12]:
ma_is.isnull().sum()

mais       0
mais112    0
mais56     0
mais28     0
mais14     0
mais7      0
mais3      0
mais1      0
dtype: int64

In [13]:
ma_is.head(10)

mais   mais112    mais56    mais28    mais14  \
item_nbr store_nbr                                                     
96995    1          0.056869  0.154255  0.172356  0.295202  0.334438   
         2          0.069360  0.161961  0.123776  0.049511  0.000000   
         3          0.096034  0.208903  0.286789  0.336299  0.375535   
         4          0.027744  0.093884  0.150635  0.099021  0.099021   
         5          0.040857  0.138257  0.202249  0.237278  0.099021   
         6          0.144969  0.375070  0.382776  0.400291  0.474556   
         7          0.090548  0.249656  0.224901  0.227004  0.247553   
         8          0.172372  0.506927  0.562372  0.720248  0.709973   
         9          0.000000  0.000000  0.000000  0.000000  0.000000   
         10         0.000000  0.000000  0.000000  0.000000  0.000000   

                       mais7     mais3     mais1  
item_nbr store_nbr                                
96995    1          0.099021  0.000000  0.000000  
         2          0.000000  0.000000  0.000000  
         3          0.454008  0.462098  0.693147  
         4          0.198042  0.231049  0.693147  
         5          0.198042  0.000000  0.000000  
         6          0.354987  0.000000  0.000000  
         7          0.099021  0.000000  0.000000  
         8          0.709973  0.963457  0.693147  
         9          0.000000  0.000000  0.000000  
         10         0.000000  0.000000  0.000000

In [14]:
del tmp,tmpg,train

Зафиксируем медианной значение этого поведения для финального расчета результата.

In [15]:
ma_is['mais']=ma_is.median(axis=1)
ma_is.reset_index(inplace=True)
ma_is.drop(list(ma_is.columns.values)[3:],axis=1,inplace=True)

In [16]:
ma_is.head(2)

,item_nbr,store_nbr,mais
0,96995,1,0.126638
1,96995,2,0.024755


Добавляем наши наработки к тестовым данным.

In [17]:
#Load test
test = pd.read_csv('data/test.csv', dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek
test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])



In [18]:
del ma_is, ma_wk, ma_dw

И наконец приступаем к итоговому расчету.

In [19]:
#Forecasting Test
test['unit_sales'] = test.mais #mais == median
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales"].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(np.expm1) # restoring unit values 

#50% more for promotion items 
test.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.5

test[['id','unit_sales']].to_csv('ma_dwof.csv.gz', index=False, float_format='%.3f', compression='gzip')

/tmp/ipykernel_6531/367965536.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test.loc[:, "unit_sales"].fillna(0, inplace=True)


## Почему используется эта формула?
1) Учет сезонности по дням недели (madw):
    * Товары часто продаются неравномерно в разные дни недели. Например, хлеб может активно продаваться в выходные, а овощи — в будние дни.
    * Использование madw позволяет скорректировать продажи с учетом дня недели.

2) Нормализация через недельное среднее (mawk):
    * Для магазинов или товаров с высокой волатильностью важно скорректировать продажи с учетом среднего недельного объема.
    * Деление на mawk помогает учитывать, насколько конкретный день (через madw) выделяется на фоне общей недели.

3) Использование базового медианного значения (mais):
    * Если товар продается стабильно в магазине, mais служит хорошим начальным приближением для прогнозов.
    * Оно учитывает скользящие средние за различные временные окна, сглаживая краткосрочные колебания.


Фильтр test['mawk'] > 0 применяется, чтобы ограничить расчеты только теми товарами и магазинами, для которых есть достоверные данные по недельным продажам.

В результате на соревновании получили следующею оценку наших вычислений: 
* Score: 0.55598
* Public score: 0.52992

Достаточно хороший результат, который нетребует сложных вычислений, кроме умственных.

*Лучший результат на текущий момент в приватной части этого соревнования 0.50918. 
https://www.kaggle.com/competitions/favorita-grocery-sales-forecasting/leaderboard?